In [1]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string

%matplotlib inline

In [4]:
# Load the Wine Dataset
df_wine =  pd.read_csv("../df_wine_vertical.csv", encoding = 'utf8', index_col=0)

In [5]:
df_wine.head(1)

,country,province,region,price,variety,points,year_of_wine
0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013


In [6]:
df_wine_sample = df_wine.sample(frac=0.10, random_state=2000)

# Data Preparition

In [7]:
df_wine = df_wine_sample

# DEFINE X and Y VARIABLES
df_wine = df_wine[df_wine['year_of_wine'] > 0]
df_X = df_wine.drop('points', axis=1)
df_X = df_X.drop('country', axis=1)
df_X = df_X.drop('price', axis=1)
df_X = df_X.drop('variety', axis=1)
df_X = df_X.drop('year_of_wine', axis=1)
df_X = df_X.drop('province', axis=1)
df_Y = df_wine[['points']]

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0)

In [8]:
def randomize(value, n):
    # makes harder to match the original value replacing n of the original chars (maybe)
    letters = string.ascii_letters
    indexes = range(0, len(value))
    indexes = random.sample(indexes, n)
    for index in indexes:
        temp = list(value)
        temp[index] = random.choice(letters)
        value = ''.join(temp)
    return value

def generate_random_typo(value):
    letters = string.ascii_letters
    degree = random.randint(1, len(value)) # add at least 1 typo
    indexes = range(0, len(value))
    indexes = random.sample(indexes, degree)

    for index in indexes:
        temp = list(value)
        temp[index] = temp[index] + random.choice(letters)
        value = ''.join(temp)

    return randomize(value, 1)

def insert_typos(df, column_name, percent): 
    ocurrence_indexes = df[column_name].index.tolist()
    columns_affected = 0
    # select random elements 
    size = len(df[column_name])
    columns_affected = int((percent * size) / 100.0)   
    selected_indexes = random.sample(ocurrence_indexes, columns_affected)
    for index, row in df.iterrows():
        if index in selected_indexes:
            value = generate_random_typo(row[0])
            df.at[index, column_name] = value
    return df

In [9]:
# GROUND TRUTH
gt_test_training = pd.concat([X_test, X_train], ignore_index=True)

In [3]:
# INTRODUCING ERRORS IN TRAINING AND TEST: SHOULD RUN SEPARETLY 

# INTRODUCE ERRORS IN TRAIN
# X_train = insert_typos(X_train, "region", 50)

# INTRODUCE ERRORS IN TEST
X_test = insert_typos(X_test, "region", 50)


NameError: name 'insert_typos' is not defined

In [11]:
# DIRTY TEST TRAINING
dirty_test_training = pd.concat([X_test, X_train], ignore_index=True)
dirty_test_training2 = X_train.copy()

In [12]:
X_test = pd.get_dummies(X_test, columns=['region'])

In [13]:
X_train = pd.get_dummies(X_train, columns=['region'])

In [14]:
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

# Precision & Recall

In [15]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(gt_test_training['region'].values, 
                                dirty_test_training['region'].values, 
                                average='weighted')

KeyError: 'country'

# HPO with ground truth

In [197]:
# # Using Randomized Search for hyperparameter optimisation

# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from scipy.stats import randint as sp_randint
# regressor3 = DecisionTreeRegressor()
# param_dist = {'max_depth': sp_randint(2,16),
#               'min_samples_split': sp_randint(2,16)}

# n_iter_search = 20
# clfrs = RandomizedSearchCV(regressor3, 
#                                    param_distributions=param_dist,
#                                    scoring='neg_mean_squared_error',
#                                    cv=5 , n_jobs=1, verbose=1,
#                                    n_iter = n_iter_search)
# clfrs.fit(df_X, df_Y)
# clfrs.best_params_, clfrs.best_score_

# Decision Tree

In [16]:
# Best result with Hyperparameter optimisation
regressor = DecisionTreeRegressor()  

# Train the model using the training sets
regressor.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regressor.predict(X_test)

# The evaluation metrics
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Mean Absolute Error: 2.2931796914661264
Mean Squared Error: 8.322005347789213
Root Mean Squared Error: 2.88478861405636
